<a href="https://colab.research.google.com/github/NikolasGialitsis/PredictingFootballMatches/blob/master/PredictingFootballOutcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!git clone https://github.com/footballcsv/greece.git


fatal: destination path 'greece' already exists and is not an empty directory.


In [90]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

main_dir = 'greece'
filename='gr.1.csv'
directories = [x[0] for x in os.walk(main_dir) if '-' in x[0]]
print(directories)
print('Number of yearly collections:',len(directories))
YearArchives = [dir + '/' + filename for dir in directories]

total=0
df = pd.DataFrame()
for id in range(len(YearArchives)):
  arch = YearArchives[id]
  if total == 0:
    df = pd.read_csv(arch)
  else:
    new_df = pd.read_csv(arch)
    df = pd.concat([df,new_df],ignore_index=True)
  print(directories[id],' contains ', df.shape[0] - total, ' football matches')
  total = df.shape[0]

print('total football matches in greece : ',total)
print(df.columns)
print(df.head())


['greece/2000-01', 'greece/2002-03', 'greece/2019-20', 'greece/2005-06', 'greece/1998-99', 'greece/2009-10', 'greece/2007-08', 'greece/2017-18', 'greece/2003-04', 'greece/2012-13', 'greece/2006-07', 'greece/2001-02', 'greece/2004-05', 'greece/2013-14', 'greece/2014-15', 'greece/1995-96', 'greece/2010-11', 'greece/2011-12', 'greece/1996-97', 'greece/1999-00', 'greece/2008-09', 'greece/2018-19', 'greece/2016-17', 'greece/1994-95', 'greece/1997-98', 'greece/2015-16']
Number of yearly collections: 26
greece/2000-01  contains  240  football matches
greece/2002-03  contains  240  football matches
greece/2019-20  contains  14  football matches
greece/2005-06  contains  240  football matches
greece/1998-99  contains  306  football matches
greece/2009-10  contains  240  football matches
greece/2007-08  contains  240  football matches
greece/2017-18  contains  240  football matches
greece/2003-04  contains  240  football matches
greece/2012-13  contains  240  football matches
greece/2006-07  con

In [91]:
# Drop 'Round' column
df = df.drop(labels = ['Round'], axis = 1)
print(df.columns)


Index(['Date', 'Team 1', 'FT', 'HT', 'Team 2'], dtype='object')


In [92]:
def count_missing_values(df):
  
  for col in df.columns:
    qmarks = df[df[col].str.contains("\?")]
    if(~qmarks.empty):
      print('\t',col,':',len(qmarks))


#check for null entries
assert(True not in df.isnull().any())


#check for missing values
print('\nMissing values initially')
count_missing_values(df)


#try to fill in missing values, between HT and FT
missingFT = df[df['FT'].str.contains("\?")].index
missingHT = df[df['HT'].str.contains("\?")].index

filled=0
for index in missingFT:
  if index not in missingHT:
    df.iloc[index]['FT'] = df.iloc[index]['HT']
    filled+=1
for index in missingHT:
  if index not in missingFT:
    df.iloc[index]['HT'] = df.iloc[index]['FT']
    filled+=1

#after correction
print('\nMissing values after filling-in %d row(s)'%filled)
count_missing_values(df)

#drop any remaining rows with missing values
missingFT = df[df['FT'].str.contains("\?")].index
df = df.drop(missingFT)
print('\nMissing values after dropping the %d non-informative row(s)'%len(missingFT))
count_missing_values(df)


Missing values initially
	 Date : 0
	 Team 1 : 0
	 FT : 2
	 HT : 1559
	 Team 2 : 0

Missing values after filling-in 1559 row(s)
	 Date : 0
	 Team 1 : 0
	 FT : 1
	 HT : 1
	 Team 2 : 0

Missing values after dropping the 1 non-informative row(s)
	 Date : 0
	 Team 1 : 0
	 FT : 0
	 HT : 0
	 Team 2 : 0


In [93]:
#ignore parentheses on team names
import re
for index, row in df.iterrows():
  team1 = row['Team 1'] 
  team2 = row['Team 2']
  
  row['Team 1'] = re.sub('\(\d*\)','',team1)
  row['Team 2'] = re.sub('\(\d*\)','',team2)
  if index == 0:
    print(team1)
    print('\t',row['Team 1'])
    print(team2)
    print('\t',row['Team 2'])
  

Olympiacos Piraeus FC (1)
	 Olympiacos Piraeus FC 
Ethnikos Asteras FC (1)
	 Ethnikos Asteras FC 


In [94]:
#repeat all matches in the dataframe, but in reverse
#ignore parentheses on team names
copy_df = df.copy()
assert(copy_df.shape[0] == df.shape[0])
for index, row in copy_df.iterrows():
  team1 = row['Team 1'] 
  team2 = row['Team 2']
  FT = row['FT']
  HT = row['HT']

  #reverse teams
  row['Team 1'] = team2
  row['Team 2'] = team1

  #reverse final score
  tokens = FT.split('-')
  row['FT'] = str(tokens[1] + '-' + tokens[0]) 

  #reverse half time score
  tokens = HT.split('-')
  row['HT'] = str(tokens[1] + '-' + tokens[0]) 

df = pd.concat([df,copy_df],ignore_index=False)
print('New merged dataframe',df.shape)

New merged dataframe (12966, 5)


In [95]:
TeamHistories = {}
for index, row in df.iterrows():
  #print('\n',row)
  team_name = row['Team 1']
  if team_name not in TeamHistories.keys():
    TeamHistories[team_name] = []
    TeamHistories[team_name].append(row)
    #print('add ',team_name)
  else:
    TeamHistories[team_name].append(row)
    #print('append ',team_name)

teams = TeamHistories.keys()
TeamGames = []
for team in teams:
  TeamGames.append(tuple((team,len(TeamHistories[team]))))

print('===== Number of games per team =====')
TeamGames.sort(key=lambda tup: tup[1],reverse=True) #sort in-place based on the number of games
for tup in TeamGames:
  team = tup[0]
  games = tup[1]
  print(team,':',games)




===== Number of games per team =====
Olympiacos Piraeus FC  : 780
Panathinaikos FC  : 780
Xanthi FC  : 780
PAOK FC  : 780
Panionios GSS  : 746
AEK Athens FC  : 712
OFI Crete FC  : 629
Aris Thessaloniki FC  : 592
Iraklis 1908 Thessaloniki FC  : 522
Ionikos FC  : 410
Levadiakos FC  : 402
Atromitos FC  : 400
Asteras Tripolis FC  : 370
AE Larissa FC  : 340
PAS Giannina FC  : 338
Apollon Smyrnis FC  : 298
Veria FC  : 290
Ergotelis FC  : 278
Kerkyra FC  : 244
Paniliakos FC  : 230
Kavala FC  : 230
Panachaiki FC  : 222
Panetolikos GFS  : 220
Panthrakikos FC  : 188
Platanias FC  : 188
Egaleo FC  : 176
Kalamata FC  : 166
Athinaikos AS FC  : 166
Proodeftiki FC  : 162
Ethnikos Piraeus FC  : 136
Ethnikos Asteras FC  : 124
Kallithea FC  : 120
Apollon Pontou FC  : 120
APO Akratitos  : 116
Edessaikos FC  : 102
AEL Kalloni FC  : 98
Heracles Almelo  : 68
Doxa Dramas FC  : 64
PAS Lamia 1964  : 62
Chalkidona FC  : 60
Panserraikos FC  : 60
Panelefsiniakos FC  : 34
Kastoria FC  : 34
Trikala FC  : 34
Niki Vo

In [119]:
#create new dataframe representation of instances that encapsulate both the current match and the recent past history of the two teams
teams = TeamHistories.keys()
columns = ['team1_g1','team1_g2','team1_g3','outcome']
data = pd.DataFrame(columns=columns)
k = 3 #number of past games that we consider relevant to the prediction
for team in teams:
  games = TeamHistories[team]
  for game_num in range(len(games)):
    date = games[game_num]['Date']
    FT = games[game_num]['FT']
    score = FT.split('-')
    diff = int(score[0])-int(score[1])
    outcome = 0
    if diff > 0:
      outcome = 1   #Won
    elif diff < 0:
      outcome = -1  #Lost
    else:
      outcome = 0   #Draw
    #get history of last k games in which the team played
    past_k_games = []
    if(game_num >= k):
      for i in range(1,k+1):
        past_score = games[game_num-i]['FT'].split('-')
        past_diff = int(past_score[0])-int(past_score[1])
        past_k_games.append(past_diff)
        past_date = games[game_num-i]['Date']
        print('\t',past_date,',final score :',games[game_num-i]['FT'],', diff :',past_diff)
    else:
      for i in range(1,k+1):
        past_k_games.append(0)


    data = data.append({'team1_g1' : past_k_games[0], 'team1_g2' : past_k_games[1], \
                        'team1_g3' : past_k_games[2] , 'outcome' : outcome},ignore_index=True)
  print(data.describe())
  break



	 (Sat) 4 Nov 2000 (W44) ,final score : 4-1 , diff : 3
	 (Sun) 1 Oct 2000 (W39) ,final score : 3-3 , diff : 0
	 (Sat) 16 Sep 2000 (W37) ,final score : 6-0 , diff : 6
	 (Sun) 19 Nov 2000 (W46) ,final score : 3-0 , diff : 3
	 (Sat) 4 Nov 2000 (W44) ,final score : 4-1 , diff : 3
	 (Sun) 1 Oct 2000 (W39) ,final score : 3-3 , diff : 0
	 (Sun) 3 Dec 2000 (W48) ,final score : 5-2 , diff : 3
	 (Sun) 19 Nov 2000 (W46) ,final score : 3-0 , diff : 3
	 (Sat) 4 Nov 2000 (W44) ,final score : 4-1 , diff : 3
	 (Sun) 17 Dec 2000 (W50) ,final score : 3-1 , diff : 2
	 (Sun) 3 Dec 2000 (W48) ,final score : 5-2 , diff : 3
	 (Sun) 19 Nov 2000 (W46) ,final score : 3-0 , diff : 3
	 (Sun) 7 Jan 2001 (W1) ,final score : 2-0 , diff : 2
	 (Sun) 17 Dec 2000 (W50) ,final score : 3-1 , diff : 2
	 (Sun) 3 Dec 2000 (W48) ,final score : 5-2 , diff : 3
	 (Sun) 21 Jan 2001 (W3) ,final score : 4-1 , diff : 3
	 (Sun) 7 Jan 2001 (W1) ,final score : 2-0 , diff : 2
	 (Sun) 17 Dec 2000 (W50) ,final score : 3-1 , diff : 2
	 (Sa